In [1]:
import json
from ast import literal_eval
from copy import copy

import pandas as pd
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer, \
    select_device
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective, \
    ClassifierTargetMapper
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN, RGCN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "variable_misuse_graph_2_percent_balanced/with_ast"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [4]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    batch_size=1,
)

In [5]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)
ntypes, etypes = dataset.get_graph_types()
config["TRAINING"]['ntypes'] = ntypes
config["TRAINING"]['etypes'] = etypes

# Declare target loading function (labels)

In [193]:
from IPython.display import Markdown
from ast import literal_eval

class SubgraphClassifierObjectiveWithSaliency(SubgraphClassifierObjective):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def _graph_embeddings(self, input_nodes, blocks, mask=None):
        graph = blocks

        emb = self._extract_embed(input_nodes, mask=mask)
        node_embs = self.graph_model(self._wrap_into_dict(emb), blocks=None, graph=graph)

        graph.nodes["node_"].data["node_embeddings"] = node_embs["node_"]

        unbatched = dgl.unbatch(graph)

        subgraph_embs = []
        for subgraph in unbatched:
            subgraph_embs.append(self.pooling_fn(subgraph.nodes["node_"].data["node_embeddings"]))

        return emb, node_embs, torch.cat(subgraph_embs, dim=0)
        
    def forward_pass_for_saliency(
            self, input_nodes, input_mask, blocks, positive_indices, negative_indices,
            update_ns_callback=None, **kwargs
    ):
        input_node_embs, output_node_embs, graph_emb = self._graph_embeddings(input_nodes, blocks, mask=input_mask)
        _, positive_emb, negative_emb, labels_pos, labels_neg = self._prepare_for_prediction(
            graph_emb, positive_indices, negative_indices, self.target_embedding_fn, update_ns_callback  # , subgraph
        )

        pos_neg_scores, avg_scores, loss  = self._compute_scores_loss(
            graph_emb, positive_emb, negative_emb, labels_pos, labels_neg
        )

        return {
            "input_nodes": input_nodes, 
            "input_node_embs": input_node_embs,
            "output_node_embs": output_node_embs,
            "graph_emb": graph_emb,
            "loss": loss,
            "prediction":  pos_neg_scores[0].argmax(dim=-1)
        }
        
    def saliency_for_batch(
            self, batch_ind, batch, optimizer
    ):
        optimizer.zero_grad()
        
        outputs = self.forward_pass_for_saliency(
            **batch
        )

        outputs["input_node_embs"].retain_grad()
        outputs["output_node_embs"]["node_"].retain_grad()
        outputs["loss"].backward()

        original_node_ids = batch["blocks"].srcnodes["node_"].data["original_id"].tolist()
        graph_node_ids = batch["blocks"].dstnodes["node_"].data["original_id"].tolist()

        input_node_embs_grad = torch.mean(outputs["input_node_embs"].grad.data.abs(), dim=-1).numpy()
        output_node_embs_grad = torch.mean(outputs["output_node_embs"]["node_"].grad.data.abs(), dim=-1).numpy()

        code_text = self.file_information[batch["indices"][0]]["filecontent"]
        misuse_span = literal_eval(self.file_information[batch["indices"][0]]["misuse_span"])

        node_info = dataset.dataset_db.database.query(f"""
        select nodes.id, name, string from nodes
        left join node_strings on node_strings.id = nodes.id
        where nodes.id in ({",".join(set(map(str, original_node_ids + graph_node_ids)))})
        """)

        node_to_name = dict(zip(node_info["id"], node_info["name"]))
        node_to_string = dict(zip(node_info["id"], node_info["string"]))

        input_node_embs_id_grad = list(zip(original_node_ids, input_node_embs_grad))
        input_node_embs_id_grad = sorted(input_node_embs_id_grad, key=lambda x: x[1], reverse=True)

        output_node_embs_id_grad = list(zip(graph_node_ids, output_node_embs_grad))
        output_node_embs_id_grad = sorted(output_node_embs_id_grad, key=lambda x: x[1], reverse=True)

        input_nodes_summary = {
            "ids": list(map(lambda x: x[0], input_node_embs_id_grad)),
            "score": list(map(lambda x: x[1], input_node_embs_id_grad)),
            "name": list(map(lambda x: node_to_name[x[0]], input_node_embs_id_grad)),
            "string": list(map(lambda x: node_to_string.get(x[0], pd.NA), input_node_embs_id_grad)),
        }

        output_nodes_summary = {
            "ids": list(map(lambda x: x[0], output_node_embs_id_grad)),
            "score": list(map(lambda x: x[1], output_node_embs_id_grad)),
            "name": list(map(lambda x: node_to_name[x[0]], output_node_embs_id_grad)),
            "string": list(map(lambda x: node_to_string.get(x[0], pd.NA), output_node_embs_id_grad)),
        }

        def print_code(code: str, misuse):
            code = code.lstrip()
            if len(misuse) == 0:
                print(code)
            else:
                display(Markdown(
                    code[:misuse[0]].replace(" ","&nbsp;") + f'<span style="color:blue">{code[misuse[0]: misuse[1]]}</span>' + code[misuse[1]: ].replace(" ","&nbsp;")
                ))

        print_code(code_text, misuse_span)
        print(f"Predicted class: {outputs['prediction']}")
        print(f"True misuse location: {misuse_span}")
        print("Input saliency")
        print(pd.DataFrame(input_nodes_summary)[["ids", "score", "name"]].head(10).to_string())
        print("Node embedding")
        print(pd.DataFrame(output_nodes_summary)[["ids", "score", "name"]].head(10).to_string())
        print("\n\n\n")


    
    def saliency(self, data_split, optimizer):
        self.file_information = {}
        with open(join(data_path, "common_filecontent.json")) as filecontents:
            for line in filecontents:
                entry = json.loads(line)
                self.file_information[entry["id"]] = entry
        
        for batch_ind, batch in enumerate(tqdm(
                self.get_iterator(data_split), total=getattr(self, f"num_{data_split}_batches")
        )):
            self.saliency_for_batch(batch_ind, batch, optimizer)
            if batch_ind == 20:
                break
            

In [194]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

In [195]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="MisuseSubgraphClassifierObjective",
                objective_class=SubgraphClassifierObjectiveWithSaliency,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
                label_loader_class=ClassifierTargetMapper,
                label_loader_params={"emb_size": None, "tokenizer_path": None, "use_ns_groups": False}
            )
        )
        
    def saliency(self):
        objective = self.objectives[0]
        objective.eval()
        objective.saliency("test", self.optimizer)

In [196]:
def compute_saliency(
        dataset, model_name, model_params, trainer_params, model_base_path,
        tokenizer_path=None, trainer=None, load_external_dataset=None
):
    model_params = copy(model_params)
    trainer_params = copy(trainer_params)

    if trainer is None:
        trainer = SamplingMultitaskTrainer

    device = select_device(trainer_params)

    trainer_params['model_base_path'] = model_base_path

    trainer = trainer(
        dataset=dataset,
        model_name=model_name,
        model_params=model_params,
        trainer_params=trainer_params,
        restore=trainer_params["restore_state"],
        device=device,
        pretrained_embeddings_path=trainer_params["pretrained"],
        tokenizer_path=tokenizer_path,
    )

    trainer.saliency()

In [197]:
compute_saliency(
    dataset, 
    model_name=RGCN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

  0%|          | 1/7483 [00:00<1:14:50,  1.67it/s]

def load_user_configuration(config_path):
    'Create a user configuration dictionary from config files\n\n    :param config_path: ``str`` path where the configuration files are kept\n    '
    user_defined_config = dict()
    user_config_file = os.path.join(config_path, 'openstack_user_config.yml')
    if os.path.isfile(user_config_file):
        with open(user_config_file, 'rb') as f:
            user_defined_config.update((yaml.safe_load(f.read()) or {
                
            }))
    base_dir = os.path.join(config_path, 'conf.d')
    if os.path.isdir(base_dir):
        _extra_config(user_defined_config, base_dir)
    if (not user_defined_config):
        raise SystemExit(('No user config loaded\nNo openstack_user_config files are available in either \n%s\nor \n%s/conf.d directory' % (config_path, config_path)))
    return user_defined_config
Predicted class: tensor([1])
True misuse location: []
Input saliency
      ids     score                                                na

def&nbsp;echo(request):
&nbsp;&nbsp;&nbsp;&nbsp;try:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;args&nbsp;=&nbsp;parser.parse(hello_args,&nbsp;request)
&nbsp;&nbsp;&nbsp;&nbsp;except&nbsp;ValidationError&nbsp;as&nbsp;err:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;json_response(<span style="color:blue">request</span>.messages,&nbsp;status=err.status_code)
&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;json_response(args)

Predicted class: tensor([1])
True misuse location: [141, 148]
Input saliency
      ids     score                                       name
0  557063  0.000324                                          _
1  720489  0.000321                 keyword_0x16f92ac20f0a4d16
2  818970  0.000290                     Try_0x16f92ac20e071131
3  219101  0.000274                                    ▁parser
4  734715  0.000237                                status_code
5   97689  0.000215               arguments_0x16f92ac20e0de47d
6  112356  0.000211                                   messages
7  888509  0.000206  hello_args@FunctionDef_0x16f92ac20eaf7963
8  907317  0.000203                                      ▁args
9  754374  0.000193                                      parse
Node embedding
      ids    score                                            name
0    3389  0.00027             args@FunctionDef_0x16f92ac20eaf7963
1   22958  0.00027                    Attribute_0x16f92ac20f74a1d6
2   97689  0.0

def&nbsp;_argcheck(self,&nbsp;a,&nbsp;b):
&nbsp;&nbsp;&nbsp;&nbsp;self.a&nbsp;=&nbsp;a
&nbsp;&nbsp;&nbsp;&nbsp;self.b&nbsp;=&nbsp;b
&nbsp;&nbsp;&nbsp;&nbsp;self.d&nbsp;=&nbsp;log(((b&nbsp;*&nbsp;1.0)&nbsp;/&nbsp;a))
&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;(((a&nbsp;>&nbsp;0)&nbsp;&&nbsp;(b&nbsp;>&nbsp;0))&nbsp;&&nbsp;(b&nbsp;>&nbsp;<span style="color:blue">b</span>))

...

  0%|          | 19/7483 [00:02<20:49,  5.97it/s]

Predicted class: tensor([1])
True misuse location: [64, 68]
Input saliency
      ids     score                                    name
0  752040  0.000571                                  ▁super
1  862631  0.000499                     _updateRegistration
2  933540  0.000434                                   ▁self
3  646203  0.000419    EPoll@FunctionDef_0x16f92ac21759ee95
4  547000  0.000378       fd@FunctionDef_0x16f92ac21759ee95
5  325005  0.000353            arguments_0x16f92ac217e2b4bd
6  846189  0.000343                                 ▁remove
7  861403  0.000338                            removeWriter
8  575580  0.000309  removeWriter@Module_0x16f92ac217f9b5a5
9  706125  0.000291     self@FunctionDef_0x16f92ac21759ee95
Node embedding
      ids     score                            name
0    1135  0.000457         Call_0x16f92ac2176c23c2
1   32704  0.000457                              ▁E
2  118936  0.000457                          Writer
3  120474  0.000457         Call_0x16f92ac

def&nbsp;variables(i,&nbsp;o):
&nbsp;&nbsp;&nbsp;&nbsp;'\n&nbsp;&nbsp;&nbsp;&nbsp;WRITEME\n\n&nbsp;&nbsp;&nbsp;&nbsp;Parameters\n&nbsp;&nbsp;&nbsp;&nbsp;----------\n&nbsp;&nbsp;&nbsp;&nbsp;i&nbsp;:&nbsp;list\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Input&nbsp;L{Variable}s.\n&nbsp;&nbsp;&nbsp;&nbsp;o&nbsp;:&nbsp;list\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Output&nbsp;L{Variable}s.\n\n&nbsp;&nbsp;&nbsp;&nbsp;Returns\n&nbsp;&nbsp;&nbsp;&nbsp;-------\n&nbsp;&nbsp;&nbsp;&nbsp;object\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The&nbsp;set&nbsp;of&nbsp;Variables&nbsp;that&nbsp;are&nbsp;involved&nbsp;in&nbsp;the&nbsp;subgraph&nbsp;that&nbsp;lies\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;between&nbsp;i&nbsp;and&nbsp;o.&nbsp;This&nbsp;includes&nbsp;i,&nbsp;o,&nbsp;orphans(i,&nbsp;o)&nbsp;and&nbsp;all&nbsp;values&nbsp;of\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;all&nbsp;intermediary&nbsp;steps&nbsp;from&nbsp;i&nbsp;to&nbsp;o.\n\n&nbsp;&nbsp;&nbsp;&nbsp;'
&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;variables_and_orphans(i,&nbsp;<span style="color:blue">i</span>)[0]

  0%|          | 20/7483 [00:02<15:37,  7.96it/s]

Predicted class: tensor([1])
True misuse location: [443, 444]
Input saliency
      ids     score                                                  name
0  557063  0.000617                                                     _
1  768745  0.000486                                               orphans
2  196208  0.000438  variables_and_orphans@FunctionDef_0x16f92ac2192aa304
3  909004  0.000433                          arguments_0x16f92ac21937cc22
4  553937  0.000357                   variables@Module_0x16f92ac219b53366
5  425004  0.000330                          Subscript_0x16f92ac2194d5932
6  964161  0.000326                             Return_0x16f92ac219f64422
7  172444  0.000319                      o@FunctionDef_0x16f92ac2192aa304
8  599037  0.000318                                                   and
9  771983  0.000317                                arg_0x16f92ac2198e9172
Node embedding
      ids    score                                                  name
0   90449  0.00056   